In [1]:
import pandas as pd
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')  # 创建一个客户端

### 连接moniter_cloudfunding数据库，或者新建数据库

In [2]:
db = client.moniter_crowdfunding  # db = client[‘test’]连接/新建test数据库

In [3]:
project = db.projects  #db['user'] # 使用/新建set01集合
frontpg = db.front_page
f_project = db.failure_projects

## 1、监测中项目

### 创建数据转换函数

In [4]:
static_fields = ['_id', '项目名称', '所属类别', '目标金额', '状态', '爬取次数', '众筹期限', 
                 '公司名称', '公司地址', '公司工作时间', '公司电话',
                 '发起人名称', '发起人链接', '状态变换时间0-1', '状态变换时间1-2']

#### (1) 获取项目静态信息

In [5]:
def get_static_proj(collections=project, category='众筹中', proj_fields = {x: True for x in static_fields}):
    items = list(collections.find({'状态': category}, projection=proj_fields))
    st_list = pd.DataFrame(items, columns=['项目名称', '所属类别', '目标金额', '状态', '爬取次数', '众筹期限', 
                                            '公司名称', '公司地址', '公司工作时间', '公司电话',
                                            '发起人名称', '发起人链接', '状态变换时间0-1', '状态变换时间1-2'],
                          index=[item['_id'] for item in items])
    return st_list

#### (2) 获取项目动态信息

In [6]:
def get_dynamic_proj(collections=project, category='众筹中'):
    dt_list = {}
    items = list(collections.find({'状态': category}))
    for item in items:
        p_id = item['_id']
        dt_list[p_id] = pd.DataFrame(item['项目动态信息'], 
                                     columns=['筹集金额', '完成百分比', '支持者数', '关注数', '点赞数', '更新时间'],
                                     index=[x['爬取时间'] for x in item['项目动态信息']])
    return dt_list                

#### (3) 打印各项目信息

In [7]:
pd_dyninfo = get_dynamic_proj()
p_ids = list(pd_dyninfo.keys())

In [8]:
pd_items = get_static_proj(category='众筹中')

In [9]:
def show_project(p_id, pd_items=pd_items, pd_dyninfo=pd_dyninfo):
    print('项目编号:', p_id, end='\t')
    print('项目名称:', pd_items['项目名称'][p_id])
    print('所属类别', pd_items['所属类别'][p_id], end='\t')
    print('众筹期限:', pd_items['众筹期限'][p_id], '天')
    return pd_dyninfo[p_id]

In [10]:
columns = ['项目名称', '状态', '爬取次数', '众筹期限', '所属类别', '状态变换时间0-1', '状态变换时间1-2']
pd_items[columns].sort_values(by='状态变换时间0-1').head()

,项目名称,状态,爬取次数,众筹期限,所属类别,状态变换时间0-1,状态变换时间1-2
98553,青白瓷旅行茶具套装,众筹中,92,60,原创设计,2018-04-28 10:11:06.367,2018-04-29 07:09:01.887
100095,荷小马绵柔极薄功能型卫生巾,众筹中,93,30,个人护理,2018-04-28 10:11:06.379,2018-05-02 10:00:15.156
99981,匠心之作-龙善堂铁壶-雨龙,众筹中,92,30,原创设计,2018-04-28 10:11:06.380,2018-04-29 10:00:22.969
98917,手绘青绿山水扇面,众筹中,93,50,原创设计,2018-04-28 10:11:06.380,2018-05-01 10:00:18.671
99680,男女通用 黑科技美牙仪,众筹中,92,50,健康医疗,2018-04-28 10:11:06.380,2018-04-28 14:00:18.670


In [34]:
p_id = '100285'
ts = show_project(p_id)
ts[30:]

项目编号: 100285	

KeyError: '100285'

In [12]:
list(project.find({'_id':'99680'}, projection={'项目动态信息.点赞数':1, '项目动态信息.关注数':1}))

[{'_id': '99680',
  '项目动态信息': [{'关注数': 88, '点赞数': 3},
   {'点赞数': 3, '关注数': 89},
   {'点赞数': 3, '关注数': 89},
   {'点赞数': 3, '关注数': 89},
   {'点赞数': 4, '关注数': 89},
   {'关注数': 92, '点赞数': 4},
   {'关注数': 97, '点赞数': 4},
   {'关注数': 98, '点赞数': 4},
   {'点赞数': 4, '关注数': 98},
   {'点赞数': 4, '关注数': 98},
   {'点赞数': 4, '关注数': 98},
   {'点赞数': 4, '关注数': 98},
   {'点赞数': 4, '关注数': 102},
   {'点赞数': 4, '关注数': 104},
   {'点赞数': 4, '关注数': 104},
   {'点赞数': 4, '关注数': 104},
   {'点赞数': 4, '关注数': 108},
   {'点赞数': 4, '关注数': 110},
   {'点赞数': 4, '关注数': 112},
   {'点赞数': 4, '关注数': 112},
   {'点赞数': 5, '关注数': 112},
   {'点赞数': 5, '关注数': 118},
   {'点赞数': 5, '关注数': 122},
   {'点赞数': 5, '关注数': 123},
   {'点赞数': 5, '关注数': 128},
   {'点赞数': 5, '关注数': 136},
   {'点赞数': 5, '关注数': 148},
   {'点赞数': 5, '关注数': 149},
   {'点赞数': 5, '关注数': 149},
   {'点赞数': 5, '关注数': 150},
   {'点赞数': 5, '关注数': 151},
   {'点赞数': 5, '关注数': 153},
   {'点赞数': 5, '关注数': 153},
   {'点赞数': 5, '关注数': 153},
   {'点赞数': 5, '关注数': 155},
   {'点赞数': 6, '关注数': 155},
   {'点赞数': 6

## 2、获取失败项目信息

In [39]:
def get_fail_proj(collections=f_project, category='众筹失败'):
    items = list(collections.find({'详细信息.状态': category}))
    d_list = []
    for item in items:
        cols = {'项目编号': item['项目编号'], '项目名称': item['详细信息']['项目名称'], 
                '开始时间': item['详细信息']['状态变换时间0-1'], '失败时间': item['失败时间'],
                '目标金额': item['详细信息']['目标金额'], '爬取次数': item['详细信息']['爬取次数'], 
                '所属类别': item['详细信息']['所属类别']}
        d_list.append(cols)
        
    d_pd = pd.DataFrame(d_list, columns=['项目编号', '项目名称', '开始时间', '失败时间', '目标金额', '爬取次数', '所属类别'], 
                        index=[i for i in range(len(d_list))])

    return d_pd

In [40]:
def get_fail_dynamic_proj(collections=f_project, category='众筹失败'):
    dt_list = {}
    items = list(collections.find({'详细信息.状态': category}))
    for i, item in enumerate(items, start=1):
        dt_list[item['项目编号']] = pd.DataFrame(item['详细信息']['项目动态信息'], 
                                     columns=['筹集金额', '完成百分比', '支持者数', '关注数', '点赞数', '更新时间'],
                                     index=[x['爬取时间'] for x in item['详细信息']['项目动态信息']])
    return dt_list  

In [43]:
get_fail_proj(category='众筹失败')

,项目编号,项目名称,开始时间,失败时间,目标金额,爬取次数,所属类别
0,99138,康风智能空气消毒净化器,2018-04-28 10:11:06.382,2018-05-03 20:30:12.496,200000,58,空气净化
1,100285,FIIT音乐AI智能跑步机,2018-05-02 19:39:24.605,2018-05-03 20:30:12.496,200000,3,健身装备
2,100465,一件超吸湿排汗的经典海魂T,2018-05-05 12:00:16.306,2018-05-05 18:00:02.730,300000,4,原创设计
3,100415,大五星枇杷 顺丰到家,2018-05-05 12:00:16.307,2018-05-06 09:57:06.568,30000,12,食品
4,98928,呆米甲醛/PM2.5监测仪,2018-05-03 20:30:30.059,2018-05-06 10:18:14.966,100000,27,智能家居
5,100281,网易味央&五芳斋匠造粽礼盒,2018-05-05 12:00:16.307,2018-05-06 10:18:14.966,50000,14,食品


In [40]:
get_fail_dynamic_proj(category='众筹未成功')

{'100040':                          筹集金额  完成百分比  支持者数  关注数  点赞数                更新时间
 2018-04-28 10:11:23.488     0    0.0     0    2    0 2018-04-27 21:39:39
 2018-04-28 11:58:49.040     0    0.0     0   12    6 2018-04-28 11:32:58
 2018-04-28 12:04:36.120     0    0.0     0   12    6 2018-04-28 11:32:58
 2018-04-28 12:06:32.528     0    0.0     0   12    6 2018-04-28 11:32:58
 2018-04-28 14:00:35.116     0    0.0     0   13    6 2018-04-28 13:34:11
 2018-04-28 16:06:53.645     0    0.0     0   17    7 2018-04-28 14:38:04
 2018-04-28 18:33:43.891     0    0.0     0   17    7 2018-04-28 18:31:53
 2018-04-28 19:08:30.519     0    0.0     0   17    7 2018-04-28 18:31:53
 2018-04-28 20:06:46.594     0    0.0     0   18    8 2018-04-28 19:53:14
 2018-04-28 20:13:37.821     0    0.0     0   18    8 2018-04-28 19:53:14
 2018-04-28 20:28:01.364     0    0.0     0   18    8 2018-04-28 19:53:14
 2018-04-28 20:29:36.850     0    0.0     0   18    8 2018-04-28 19:53:14
 2018-04-28 21:48:05.994    

In [28]:
dyn_fail = get_fail_dynamic_proj()
fail_proj = get_fail_proj(collections=f_project)

In [33]:
fail_proj

,项目编号,项目名称,开始时间,失败时间,目标金额,爬取次数,所属类别
0,99138,康风智能空气消毒净化器,2018-04-28 10:11:06.382,2018-05-03 20:30:12.496,200000,58,空气净化
1,100285,FIIT音乐AI智能跑步机,2018-05-02 19:39:24.605,2018-05-03 20:30:12.496,200000,3,健身装备


In [41]:
i = 1
p_id = fail_proj['项目编号'][i]
p_name = fail_proj['项目名称'][i]
p_times = fail_proj['爬取次数'][i]
p_cat = fail_proj['所属类别'][i]
print(p_id, p_name, p_times, p_cat)
dyn_fail[p_id]

100285 FIIT音乐AI智能跑步机 3 健身装备


,筹集金额,完成百分比,支持者数,关注数,点赞数,更新时间
2018-05-02 19:40:09.445,0,0.0,0,0,0,2018-05-02 19:40:10.359
2018-05-02 20:00:57.578,0,0.0,0,0,0,2018-05-02 20:00:58.219
2018-05-02 22:01:00.646,0,0.0,0,6,0,2018-05-02 21:46:22.000


In [42]:
project.find_one({'_id': '100040'})

## 3、将失败项目重新设置为成功状态

In [65]:
import datetime

In [36]:
fail_projs = list(f_project.find({}))
for proj in fail_projs:
    x = proj['详细信息']
    p_id = x['_id']
    try:
        project.insert_one(x)
        project.update_one({'_id': p_id}, {'$set': {'状态': '众筹中'}})
        print(p_id, '数据转移成功')
    except Exception as e:
        print(e)

E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "99138" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100285" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100040" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100465" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100415" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "99706" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "98928" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100281" }
E11000 duplicate key error collection: moniter_crowdfunding.projects index: _id_ dup key: { : "100051" }
E11000 duplicate key error collection: moniter_crowdfundin

In [70]:
fail_projs = list(f_project.find({}))

In [71]:
for x in fail_projs:
    print(x['项目编号'], x['详细信息']['项目名称'], x['详细信息']['状态'])

100285 FIIT音乐AI智能跑步机 众筹失败
100465 一件超吸湿排汗的经典海魂T 众筹失败


In [47]:
f_ids = [(x['项目编号'], x['详细信息']['项目名称']) for x in fail_projs]

In [50]:
for p_id, p_name in f_ids:
    try:
        x = project.find_one({'_id': p_id}, projection={'_id':True, '项目名称':True})
        if p_name == x['项目名称']:
            f_project.delete_one({'项目编号': p_id})
            print(p_id, p_name, '，从failure_projects中删除！')
        else:
            print('的确是众筹失败项目:', p_id)
    except Exception as e:
        print(p_id, e)

99138 康风智能空气消毒净化器 ，从failure_projects中删除！
100285 'NoneType' object is not subscriptable
100040 iBranco上上筷礼盒 ，从failure_projects中删除！
100465 'NoneType' object is not subscriptable
100415 大五星枇杷 顺丰到家 ，从failure_projects中删除！
99706 扶贫普洱茶-助力佤族脱贫 ，从failure_projects中删除！
98928 呆米甲醛/PM2.5监测仪 ，从failure_projects中删除！
100281 网易味央&五芳斋匠造粽礼盒 ，从failure_projects中删除！
100051 观垚茶器-紫砂六师共铸传承 ，从failure_projects中删除！
99138 康风智能空气消毒净化器 ，从failure_projects中删除！
100285 'NoneType' object is not subscriptable
100465 'NoneType' object is not subscriptable
100285 'NoneType' object is not subscriptable
100465 'NoneType' object is not subscriptable
